In [1]:
import pandas as pd 
df = pd.read_csv("cc3m_full_with_path.csv")

In [2]:
df

,url,key,status,width,height,exif,caption,alt_txt,path
0,https://ak0.picdn.net/shutterstock/videos/3380...,1470004,success,852.0,480.0,{},the effect of moving the large colored particl...,"""""",/home/temp/Desktop/ccm3_full/00147/001470004.jpg
1,https://ak9.picdn.net/shutterstock/videos/2377...,1470024,success,852.0,480.0,{},wing of an airplane passing over clouds at hig...,"""""",/home/temp/Desktop/ccm3_full/00147/001470024.jpg
2,https://ak7.picdn.net/shutterstock/videos/1292...,1470042,success,852.0,480.0,{},man holding the keys to her new car in a motor...,"""""",/home/temp/Desktop/ccm3_full/00147/001470042.jpg
3,https://ak9.picdn.net/shutterstock/videos/1147...,1470044,success,852.0,480.0,{},background with particles and stars .,"""""",/home/temp/Desktop/ccm3_full/00147/001470044.jpg
4,https://ak8.picdn.net/shutterstock/videos/5176...,1470039,success,852.0,480.0,{},food canned in mason jars on shelves in a stor...,"""""",/home/temp/Desktop/ccm3_full/00147/001470039.jpg
...,...,...,...,...,...,...,...,...,...
2555423,http://l7.alamy.com/zooms/5f24d20339df4077a506...,2199920,success,640.0,447.0,{},senior woman in the park,"""""",/home/temp/Desktop/ccm3_full/00219/002199920.jpg
2555424,http://l7.alamy.com/zooms/7c2ad41134dc4b1984d9...,2199914,success,640.0,446.0,{},remote house in the forest by lake,"""""",/home/temp/Desktop/ccm3_full/00219/002199914.jpg
2555425,http://l7.alamy.com/zooms/0dd49ace792a4db6be39...,2199999,success,640.0,448.0,{},wind blown tree on the beach,"""""",/home/temp/Desktop/ccm3_full/00219/002199999.jpg
2555426,http://l7.alamy.com/zooms/28f1e935be484181a90c...,2199994,success,640.0,447.0,{},snow covered city park in a winter day .,"""""",/home/temp/Desktop/ccm3_full/00219/002199994.jpg


In [11]:
from  textwrap import TextWrapper
import textwrap
txt = "HEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEELLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOWORLD IS THIS THE REAL"
txt = TextWrapper(width=27, break_long_words=True)

In [13]:
txt.

True

In [2]:
from dataset import * 
from tqdm import tqdm 

In [3]:
dataset = CC3M()

In [11]:
import torch 
replace_idx = torch.randint(17_000, (1, 1))

In [13]:
replace_idx.item()

9224

In [4]:
for i in tqdm(range(len(dataset))): 
    _ = dataset[i]

  1%|          | 17397/2555428 [05:02<12:14:37, 57.58it/s]


KeyboardInterrupt: 

In [42]:
from torchvision import datasets, transforms
transform = transforms.Compose([
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
        transforms.Resize((224, 224))
    ])
dataset = CC3M(transform, transform)

In [ ]:
dataset[]

In [22]:
from PIL import ImageFont, ImageDraw, Image 

def render_text(txt:str, image_size: int=224, font_size: int = 16, max_chars=768,
                       background_brightness=127, text_brightness=0,
                       lower=True, monospace=False, spacing=1, min_width=4,
                       resize_method="area", max_width=28):
    if len(txt)> max_chars:
        txt = txt[:max_chars]
    if lower: 
        txt = txt.lower() 
    wrapper = textwrap.TextWrapper(width=max_width)
    lines = wrapper.wrap(txt) 
    new_txt = ""
    for line in lines: 
        new_txt+= line+'\n'
    image = Image.new("RGBA", (image_size*3,image_size*3), (background_brightness,background_brightness,background_brightness))
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("/home/temp/Desktop/CLIPPO/unifont-15.0.06.otf", font_size*3)
    draw.text((0, 0), new_txt, (text_brightness,text_brightness,text_brightness), font=font, spacing=spacing)
    img_resized = image.resize((image_size,image_size), Image.ANTIALIAS)
    return img_resized


In [29]:
import torch 
from network import CLIPPO
clippo = CLIPPO(1)
clippo = clippo.cuda() 
#clippo.load_state_dict(torch.load("/home/temp/Desktop/KD/CLIPPO/clippo.pt")) 

In [30]:
image1, text1 = dataset[0]
image2, text2 = dataset[2]

In [31]:
clippo(image1[None, ].cuda(), text1[None, ].cuda())

tensor(0., device='cuda:0', grad_fn=<MeanBackward0>)

In [32]:
text_embed = clippo.encoder(torch.stack([text1, text2]).cuda())
image_embed = clippo.encoder(torch.stack([image1, image2]).cuda())

In [37]:
image_embed

tensor([[ 1.0033,  0.2610, -1.0145,  ...,  2.3434,  0.7584,  0.9603],
        [-0.0209,  0.7269, -1.5723,  ...,  1.6124,  0.9936,  1.5815]],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [38]:
text_embed

tensor([[ 0.2167,  1.2422,  0.9508,  ..., -1.8052,  0.0664,  0.9264],
        [ 0.2383,  1.1640,  0.9022,  ..., -1.8645,  0.0767,  0.9156]],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [40]:
torch.cosine_similarity(text_embed[1][None, ], text_embed[0][None, ])

tensor([0.9985], device='cuda:0', grad_fn=<SumBackward1>)

torch.Size([2, 768])

In [47]:
torch.isnan(dataset[499][0]).any()

tensor(False)

In [ ]:
import numpy as np 


In [ ]:
dataset 